### Day 2 - Part B
Find all numbers in the input range that is made of n identical squences.

In [50]:
from itertools import product

from util.aoc_utility import *

DAY = 2
TEST_MODE = 0
SECTIONS = False

data_in = load_input(day=DAY, test_mode=TEST_MODE, sections=SECTIONS)
ranges = line_to_arr(data_in[0], ",")

In [51]:
ranges[0:3]

['4487-9581', '755745207-755766099', '954895848-955063124']

In [52]:
def alg_combinations_for_n(n):
    """Returns every algebraic combination for n digits.

    Args:
    n (int): Number of digits

    Returns:
    combos (arr): Array of algebraic combinations that represents every pattern 
    that can be made with unique digits.

    For a pattern like ABCB, this means A, B, C are digits 0-9 where A != B != C.
    """

    #Unique digits that represent different digits
    #A-J to represent 10 possible values (0-9)
    unique_digits = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']

    #Recursive function to get all combinations
    combos = get_next_combinations(n, 0, unique_digits)

    return combos
    
def get_next_combinations(n, used_digits, unique_digits):
    """Recursive function to get all combinations of algebraic combinations.

    Args:
    n (int): Number of remaining digits
    used_digits (int): How many unique digits used so far
    unique_digits (arr): List of algebra characters available for combinations

    Returns:
    combos (arr): Array of algebra combinations made using unique_digits
    """

    #Get list of available digits (B can only be used if A has been used, C can 
    #only be used if B has been used, etc.)
    available_digits = unique_digits[0:used_digits+1]

    #If only 1 remaining digit then return all available
    if n == 1:
        return available_digits
    
    #Apply recursion
    else:
        #For both the situation of introducing a new digit and reusing a previously used one,
        # recursively get all possible tails
        tails_new_digit = get_next_combinations(n-1, used_digits + 1, unique_digits)
        tails_reuse = get_next_combinations(n-1, used_digits, unique_digits)

        combos = []
        
        #For each available digit, get all combinations with tails
        for digit in available_digits:

            #The last available_digit is not previously used so different tails
            if digit == available_digits[-1]:
                combos += [digit + tail for tail in tails_new_digit]
            else:
                combos += [digit + tail for tail in tails_reuse]

        return combos


In [53]:
def all_pattern_maps():
    """Returns a dictionary that contains an array of every possible character map

    Returns:
    pattern_map (dict): Dictionary with keys 1 to 10 that represent the number of unique characters in a pattern. 
     Each key maps to an array containing every mapping combination
    """

    pattern_map = {}

    for i in range(10):
        pattern_map[str(i + 1)] = get_all_maps_for_n(i+1, first=True)

    return pattern_map

def get_all_maps_for_n(n, remaining_values=list(range(0, 10)), first=False):
    """For n values, get all numeric mappings where the first digit is 1-9 and 
    all are unique
    
    Args:
    n (int): Number of unique values to map to numeric values
    remaining_values (arr): List of unallocated values
    first (bool): Flag to say if this is the first call of the function 
     (since the first digit cannot be 0)

    Returns:
    maps (arr): Array of all numeric maps
    """

    #If first call of function, restrict num_range to not include 0
    if first:
        num_range = list(range(1, 10))
    else:
        num_range = remaining_values

    #If down to last value, just use all values in num_range
    if n == 1:
        return [[x] for x in num_range]
    
    #Otherwise, recursively get all combinations
    else:
        maps = []
        for idx, option in enumerate(num_range):
            #Remove the selected option from recursive calls to stop it being reused
            remaining_values = num_range[:idx] + num_range[idx+1:]
            if first:
                remaining_values = [0] + remaining_values
                
            #Get all recursive maps
            maps += [[option] + x for x in get_all_maps_for_n(n-1, remaining_values)]

        return maps

In [54]:
def apply_pattern_map(pattern, pattern_map, chars=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']):
    """For a given pattern, translate it to all possible numeric values

    Args:
        pattern (str): Algebraic string pattern
        pattern_map (dict): Dictionary containing all pattern_map options
        chars (arr): List of possible characters in the pattern string

    Returns:
        numbers (arr): List of all numbers that can be made with this pattern string
    """
    #Get number of unique characters in pattern
    n = len(''.join(set(pattern)))

    #Build array of valid numbers that match the pattern
    numbers = []
    for maps in pattern_map[str(n)]:
        #Convert the current pattern into the numeric equivalent
        cur = pattern
        for i in range(n):
            cur = cur.replace(chars[i], str(maps[i]))

        numbers.append(int(cur))

    return numbers

def limit_to_range(x, y, numbers):
    """Limit a list of numbers to x and y

    Args:
        x (int): Lower bound
        y (int): Upper bound
        numbers (arr): List of numbers

    Returns:
        valid_numbers (arr): List of numbers within bounds
    """
    valid_numbers = [n for n in numbers if x <= n and n <= y]

    return valid_numbers

In [55]:
def factors_of_n(n):
    """Get all factors of n
    Args:
    n (int): Target number
    
    Returns:
    factors (arr): List of factors
    """
    
    factors = []
    for i in range(1, n):
        if n % i == 0:
            factors.append(i)

    return factors

In [56]:
#Build the pattern map
pattern_map = all_pattern_maps()

In [ ]:
total = 0

for range_pair in ranges:
    #For a given range, get the lower and upper bounds
    range_arr = range_pair.split("-")
    lower_bound = int(range_arr[0])
    upper_bound = int(range_arr[1])

    lower_n = len(range_arr[0])
    upper_n = len(range_arr[1])

    numbers = []

    for n in range(lower_n, upper_n+1):

        #Get all factors of n
        factors = factors_of_n(n)

        #For each factor (number of sequences in the pattern) get all solutions
        for f in factors:
            patterns = alg_combinations_for_n(f)

            all_numbers = []
            for pat in patterns:
                all_numbers += apply_pattern_map(pat, pattern_map)

            all_full_numbers = [int(str(p) * int(n/f)) for p in all_numbers]

            res = limit_to_range(lower_bound, upper_bound, all_full_numbers)
        
            numbers += res

    #Combine solutions into a set to remove duplicate solutions
    total += sum(set(numbers))

print(total)

31755323497
